# pilot test3
## - main protocol test
## - 32ch
## - ch1: empty, ch33: empty --> 둘다 삭제
## - txt2mat
### 20220126 작성 시작


### 20220126 작성 시작

In [4]:
import os 
import numpy as np
import natsort
import pandas as pd
from tqdm.notebook import tqdm, trange
import scipy.io as spio
%matplotlib inline

In [9]:
#default_path = 'E:\\B2X\\2차년도\\03_pilot\\subject_data'
default_path = 'C:/Users/USER/Desktop/tmp/txt'
default_path

'C:/Users/USER/Desktop/tmp/txt'

## subject list 확보

In [10]:
def OnlyFolder(path):
    # return only folder, not file
    # path: default_path which contains subject folders
    
    dir_list = os.listdir(path) # get all directory below the path
    output = []
    for i in range(len(dir_list)):
        temp_path = os.path.join(path, dir_list[i])
        if os.path.isdir(temp_path) == True:
            output.append(dir_list[i])
    return output

In [11]:
total_subject_list = OnlyFolder(default_path)
total_subject_list = natsort.natsorted(total_subject_list) # 번호 순서대로 정렬
total_subject_list

['20211113_KDH',
 '20211123_HHB',
 '20211229_PSR',
 '20211230_CJK',
 '20220104_CKM',
 '20220118_KDH',
 '20220119_PSR',
 '20220120_HHB',
 '20220121_CKM',
 '20220124_CSJ',
 '20220125_KMS',
 '20220126_CJK',
 '20220126_CSJ',
 '20220127_KMS',
 '20220203_LJH',
 '20220203_PBJ',
 '20220204_PBJ',
 '20220207_SYH',
 '20220208_SYH',
 '20220209_LJH']

## 분석할 subject 선택

In [12]:
choose_sub = range(len(total_subject_list))
sub_list = []
for sub_num in range(len(choose_sub)):    
    sub_list.append(total_subject_list[choose_sub[sub_num]])
    
print('Target Subjects: {}'.format(sub_list))

Target Subjects: ['20211113_KDH', '20211123_HHB', '20211229_PSR', '20211230_CJK', '20220104_CKM', '20220118_KDH', '20220119_PSR', '20220120_HHB', '20220121_CKM', '20220124_CSJ', '20220125_KMS', '20220126_CJK', '20220126_CSJ', '20220127_KMS', '20220203_LJH', '20220203_PBJ', '20220204_PBJ', '20220207_SYH', '20220208_SYH', '20220209_LJH']


In [6]:
def SelectTxtFiles(file_list):
    # file_list 중, 'txt' 문자열을 포함한 것만 남김
    # file_list : 검사하고자 하는 file 이름이 담긴 list
    print("\nTest... if txt or not")
    result = []
    for i in range(len(file_list)):
        if file_list[i].endswith(".txt"):
            result.append(file_list[i])
    print('Filtered txt files: {}'.format(result))
    return result


def TxtRead(path, filename):
    # txt 파일을 line별로 읽어서 list로 반환
    # path: 파일이 저장된 경로
    # filname : 파일 이름

    file_path = path + '\\' + filename
    
    result = []
    num = 0
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            data = line.strip().split()
            result.append(data)
    return result

In [16]:
for sub_num in tqdm(range(len(choose_sub)), desc='Total Processing'):
    
    print('------------------------------------')
    print(sub_list[sub_num])
    # 현재 sub_num에 해당하는 subject의 txt 파일이 존재하는 경로
    subject_txt_path = default_path + '\\' + sub_list[sub_num] + '\\EEG'
    
    # txt 경로에서, 확장자명이 txt인 파일들만 선택
    txt_list = SelectTxtFiles(os.listdir(subject_txt_path))
    print('\nData Extraction Start ...')
    
    # 5개의 txt 파일에 대해서 각각 data를 추출하고, mat파일로 저장
    for stim_num in tqdm(range(len(txt_list)), desc=sub_list[sub_num] + ' Processing'):
        print('{}th txt'.format(stim_num + 1), end='.. ')
        
        temp_data = TxtRead(subject_txt_path, txt_list[stim_num])
        header = temp_data[:3] # header 성분
        non_header = temp_data[3:] # non-header --> EEG data
    
        # non_header의 마지막 row에는 빈 배열이 존재하기 때문에, numpy로 전환하면서 오류가 발생함
        # --> 마지막 row를 삭제
        if len(non_header[len(non_header)-1]) == 0:
            del non_header[len(non_header)-1] # list의 요소 제거 방법인 del을 사용
            # non_header list를 numpy array로 변환
            non_header = np.array(non_header, dtype=float)
        non_header_df = pd.DataFrame(non_header)
        
        # useless channel 제거
        # ch1 : time 정보 --> 제거
        # ch2 : noise --> 제거
        # ch34 : flag --> 제거
        # 총 34개 채널 중, 3개 채널을 제거하면 31 채널이 남는다
        if non_header_df.shape[1] == 34:
            # ch34가 존재하는 경우
            non_header_df = non_header_df.drop([0, 1, 33], axis=1)
        else:
            # ch34가 존재하지 않으면, ch1과 ch2만 index 0,1을 이용하여 제거            
            non_header_df = non_header_df.drop([0, 1], axis=1)
            
        print(" Shape: {}, Size: {}".format(non_header_df.shape, non_header_df.size), end=' --> Save...')
        
        data = np.array(non_header_df)
        save_as = os.path.join(subject_txt_path, txt_list[stim_num][:-3]+'mat')
        spio.savemat(save_as, mdict={'data': data})
        print("done !")

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html